In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings 
warnings.filterwarnings(action='ignore')


import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, LSTM, InputLayer
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn import metrics 
from tensorflow import keras
import random  
from tensorflow.keras.callbacks import EarlyStopping

seed_num = 42
# os.environ["CUDA_VISIBLE_DEVICES"] = '-1'
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        for i in range(len(gpus)):
            tf.config.experimental.set_memory_growth(gpus[i], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)
gpus = tf.config.experimental.list_physical_devices('GPU')
seed_num = 42
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

X_train.shape, y_train.shape, X_test.shape, y_test.shape


def get_model():
    lstm = Sequential()
    lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
    lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(Dropout(0.2))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(units=1, activation='sigmoid'))

    lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
                 loss = "binary_crossentropy", metrics=['acc'])
    return lstm

from keras.wrappers.scikit_learn import KerasClassifier

class MyKerasClassifier(KerasClassifier):
    def fit(self, x, y, sample_weight=None, **kwargs):
        y = np.array(y)
        if len(y.shape) == 2 and y.shape[1] > 1:
            self.classes_ = np.arange(y.shape[1])
        elif (len(y.shape) == 2 and y.shape[1] == 1) or len(y.shape) == 1:
            self.classes_ = np.unique(y)
            y = np.searchsorted(self.classes_, y)
        else:
            raise ValueError('Invalid shape for y: ' + str(y.shape))
        self.n_classes_ = len(self.classes_)
        #---------------수정---------------
        if sample_weight is not None:
            print('sample weight : ', sample_weight)
            if sample_weight[0] == 0.00016175994823681658:
                print('x, y', x.shape, x.sum().sum())
                return super(MyKerasClassifier, self).fit(x, y, **kwargs)
            weights = sample_weight / sum(sample_weight)
            random_range = [(sum(weights[:i]), sum(weights[:i])+weights[i]) if i!=0 else (0, weights[i]) for i in range(len(weights))]
            random_nums = [random.random() for _ in range(len(weights))]
            idx_list = []
            for i in random_nums:
                for j in random_range:
                    if j[0] < i <= j[1]:
                        idx_list.append(random_range.index(j))
                        break
            new_x = x[idx_list, :, :]
            new_y = y[idx_list]
            print(new_x.sum().sum())
            print('new_x, new_y', new_x.shape, new_y.shape)
            return super(MyKerasClassifier, self).fit(new_x, new_y, **kwargs)
        
    def predict(self, x, **kwargs):
        return super(MyKerasClassifier, self).predict(x)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [10]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings 
warnings.filterwarnings(action='ignore')


import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, LSTM, InputLayer
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn import metrics 
from tensorflow import keras
import random  
from tensorflow.keras.callbacks import EarlyStopping

seed_num = 42
# os.environ["CUDA_VISIBLE_DEVICES"] = '-1'
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        for i in range(len(gpus)):
            tf.config.experimental.set_memory_growth(gpus[i], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)
gpus = tf.config.experimental.list_physical_devices('GPU')
seed_num = 42
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

X_train.shape, y_train.shape, X_test.shape, y_test.shape


def get_model():
    lstm = Sequential()
    lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
    lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(Dropout(0.2))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(units=1, activation='sigmoid'))

    lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
                 loss = "binary_crossentropy", metrics=['acc'])
    return lstm

from keras.wrappers.scikit_learn import KerasClassifier
import random
class MyKerasClassifier(KerasClassifier):
    def fit(self, x, y, sample_weight=None, validation_split=0.25, **kwargs):
        y = np.array(y)
        if len(y.shape) == 2 and y.shape[1] > 1:
            self.classes_ = np.arange(y.shape[1])
        elif (len(y.shape) == 2 and y.shape[1] == 1) or len(y.shape) == 1:
            self.classes_ = np.unique(y)
            y = np.searchsorted(self.classes_, y)
        else:
            raise ValueError('Invalid shape for y: ' + str(y.shape))
        self.n_classes_ = len(self.classes_)
        #---------------수정---------------
        if sample_weight is not None:
            print('sample weight : ', sample_weight)
            if sample_weight[0] == 0.00016175994823681658:
                print('x, y', x.shape, x.sum().sum())
                return super(MyKerasClassifier, self).fit(x, y, **kwargs)
            print('x sum', x.sum().sum())
            
            idx = np.arange(len(x))
            random.shuffle(idx)
            i = int(len(x)*0.75)
            train_x, train_y = x[idx[:i],:,:], y[idx[:i]]
            val_x, val_y = x[idx[i:],:,:], y[idx[i:]]
            
            
            train_sw, val_sw = sample_weight[idx[:i]], sample_weight[idx[i:]]
            train_sw, val_sw = train_sw/sum(train_sw)*len(train_sw), val_sw/sum(val_sw)*len(val_sw)
            
            weights = val_sw / sum(val_sw)
            random_range = [(sum(weights[:i]), sum(weights[:i])+weights[i]) if i!=0 else (0, weights[i]) for i in range(len(weights))]
            random_nums = [random.random() for _ in range(len(weights))]
            idx_list = []
            for i in random_nums:
                for j in random_range:
                    if j[0] < i <= j[1]:
                        idx_list.append(random_range.index(j))
                        break
            new_val_x = val_x[idx_list, :, :]
            new_val_y = val_y[idx_list]
            
            kwargs['validation_data'] = (new_val_x, new_val_y)
            kwargs['sample_weight'] = train_sw
            return super(MyKerasClassifier, self).fit(train_x, train_y, **kwargs)
        
        return super(MyKerasClassifier, self).fit(x, y, **kwargs)
    
    
    
    def predict(self, x, **kwargs):
        return super(MyKerasClassifier, self).predict(x)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


# Adaboost-LSTM

## Single LSTM

In [36]:
X_train.shape

(6182, 10, 4068)

In [32]:
idx = np.arange(7727)
random.shuffle(idx)
idx

array([7267, 6914,  990, ..., 2575, 3450, 1696])

In [6]:
with tf.device('/device:GPU:0'):
    print("Single LSTM Start")
    model = get_model()
    
    early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)
    cb_checkpoint = ModelCheckpoint(filepath='./models/single_lstm.h5', monitor='val_loss',
                                    verbose=1, save_best_only=True)
    model.fit(X_train, y_train, epochs=50, batch_size=128, validation_split=0.25, callbacks=[early_stop, cb_checkpoint])
    preds = model.predict(X_test)

    preds[preds>0.5]=1
    preds[preds<=0.5]=0
    precision = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    f1 = f1_score(y_test, preds)
    roc_auc = roc_auc_score(y_test, preds)
    acc = accuracy_score(y_test, preds)

    print(f'accuracy : {acc}, precision : {precision}, recall : {recall}, f1 : {f1}, roc_auc : {roc_auc}')

Single LSTM Start
Epoch 1/50
37/37 [==============================] - 6s 81ms/step - loss: 0.6736 - acc: 0.6055 - val_loss: 0.6657 - val_acc: 0.6177
Epoch 2/50
37/37 [==============================] - 3s 70ms/step - loss: 0.6696 - acc: 0.6100 - val_loss: 0.6509 - val_acc: 0.6177
Epoch 3/50
37/37 [==============================] - 3s 70ms/step - loss: 0.6045 - acc: 0.6706 - val_loss: 0.5288 - val_acc: 0.7510
Epoch 4/50
37/37 [==============================] - 3s 70ms/step - loss: 0.4900 - acc: 0.7821 - val_loss: 0.4942 - val_acc: 0.7639
Epoch 5/50
37/37 [==============================] - 3s 71ms/step - loss: 0.4187 - acc: 0.8261 - val_loss: 0.4894 - val_acc: 0.7743
Epoch 6/50
37/37 [==============================] - 3s 70ms/step - loss: 0.3760 - acc: 0.8503 - val_loss: 0.5036 - val_acc: 0.7743
Epoch 7/50
37/37 [==============================] - 3s 71ms/step - loss: 0.3591 - acc: 0.8613 - val_loss: 0.5329 - val_acc: 0.7587
Epoch 8/50
37/37 [==============================] - 3s 71ms/step 

In [ ]:
%%time
from tensorflow.keras.callbacks import ModelCheckpoint

with tf.device('/device:GPU:0'):
    early_stop = EarlyStopping(monitor='val_loss', patience=50, verbose=1, restore_best_weights=True)
#     cb_checkpoint = ModelCheckpoint(filepath='./models/adaboost_lstm.h5', monitor='val_acc',
#                                     verbose=1, save_best_only=True)
    base_estimator = MyKerasClassifier(build_fn=get_model, epochs=300, batch_size=10000,
                                       validation_split=0.25, callbacks=[early_stop])
#     base_estimator = MyKerasClassifier(build_fn=get_model, epochs=50, batch_size=128, validation_split=0.25)

    boosted_classifier = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=5, random_state=42, learning_rate=1)
    
    print("Adaboost LSTM Start")
    boosted_classifier.fit(X_train, y_train)
    preds = boosted_classifier.predict(X_test)

    precision = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    f1 = f1_score(y_test, preds)
    roc_auc = roc_auc_score(y_test, preds)
    acc = accuracy_score(y_test, preds)

    print(f'Adaboost accuracy : {acc}, precision : {precision}, recall : {recall}, f1 : {f1}, roc_auc : {roc_auc}')

Adaboost LSTM Start
sample weight :  [0.00016176 0.00016176 0.00016176 ... 0.00016176 0.00016176 0.00016176]
x, y (6182, 10, 4068) 1644947.0
Epoch 1/300
1/1 [==============================] - 27s 27s/step - loss: 0.6860 - acc: 0.5528 - val_loss: 0.6703 - val_acc: 0.6177
Epoch 2/300
1/1 [==============================] - 2s 2s/step - loss: 0.6799 - acc: 0.5796 - val_loss: 0.6663 - val_acc: 0.6177
Epoch 3/300
1/1 [==============================] - 3s 3s/step - loss: 0.6785 - acc: 0.5936 - val_loss: 0.6652 - val_acc: 0.6177
Epoch 4/300
1/1 [==============================] - 3s 3s/step - loss: 0.6767 - acc: 0.6085 - val_loss: 0.6657 - val_acc: 0.6177
Epoch 5/300
1/1 [==============================] - 3s 3s/step - loss: 0.6808 - acc: 0.6098 - val_loss: 0.6664 - val_acc: 0.6177
Epoch 6/300
1/1 [==============================] - 2s 2s/step - loss: 0.6788 - acc: 0.6100 - val_loss: 0.6667 - val_acc: 0.6177
Epoch 7/300
1/1 [==============================] - 3s 3s/step - loss: 0.6761 - acc: 0.609

1/1 [==============================] - 2s 2s/step - loss: 0.6510 - acc: 0.6130 - val_loss: 0.6419 - val_acc: 0.6177
Epoch 59/300
1/1 [==============================] - 3s 3s/step - loss: 0.6484 - acc: 0.6119 - val_loss: 0.6378 - val_acc: 0.6177
Epoch 60/300
1/1 [==============================] - 3s 3s/step - loss: 0.6409 - acc: 0.6193 - val_loss: 0.6333 - val_acc: 0.6177
Epoch 61/300
1/1 [==============================] - 2s 2s/step - loss: 0.6377 - acc: 0.6182 - val_loss: 0.6284 - val_acc: 0.6177
Epoch 62/300
1/1 [==============================] - 3s 3s/step - loss: 0.6313 - acc: 0.6221 - val_loss: 0.6230 - val_acc: 0.6177
Epoch 63/300
1/1 [==============================] - 3s 3s/step - loss: 0.6276 - acc: 0.6251 - val_loss: 0.6172 - val_acc: 0.6177
Epoch 64/300
1/1 [==============================] - 2s 2s/step - loss: 0.6208 - acc: 0.6322 - val_loss: 0.6110 - val_acc: 0.6177
Epoch 65/300
1/1 [==============================] - 3s 3s/step - loss: 0.6159 - acc: 0.6346 - val_loss: 0.6042

1/1 [==============================] - 3s 3s/step - loss: 0.2494 - acc: 0.9308 - val_loss: 0.6405 - val_acc: 0.7607
Epoch 122/300
1/1 [==============================] - 2s 2s/step - loss: 0.2451 - acc: 0.9305 - val_loss: 0.6436 - val_acc: 0.7555
Epoch 123/300
1/1 [==============================] - 2s 2s/step - loss: 0.2430 - acc: 0.9316 - val_loss: 0.6523 - val_acc: 0.7484
Epoch 124/300
1/1 [==============================] - 3s 3s/step - loss: 0.2438 - acc: 0.9325 - val_loss: 0.6539 - val_acc: 0.7555
Epoch 125/300
1/1 [==============================] - 3s 3s/step - loss: 0.2392 - acc: 0.9340 - val_loss: 0.6583 - val_acc: 0.7536
Epoch 126/300
1/1 [==============================] - 3s 3s/step - loss: 0.2410 - acc: 0.9323 - val_loss: 0.6669 - val_acc: 0.7484
Epoch 127/300
1/1 [==============================] - 2s 2s/step - loss: 0.2388 - acc: 0.9340 - val_loss: 0.6665 - val_acc: 0.7549
Epoch 128/300
1/1 [==============================] - 2s 2s/step - loss: 0.2339 - acc: 0.9359 - val_loss:

1/1 [==============================] - 2s 2s/step - loss: 0.6959 - acc: 0.5186 - val_loss: 0.6908 - val_acc: 0.6195
Epoch 48/300
1/1 [==============================] - 2s 2s/step - loss: 0.6994 - acc: 0.5091 - val_loss: 0.6908 - val_acc: 0.6195
Epoch 49/300
1/1 [==============================] - 2s 2s/step - loss: 0.6973 - acc: 0.5007 - val_loss: 0.6908 - val_acc: 0.6195
Epoch 50/300
1/1 [==============================] - 3s 3s/step - loss: 0.6975 - acc: 0.5088 - val_loss: 0.6908 - val_acc: 0.6195
Epoch 51/300
1/1 [==============================] - 2s 2s/step - loss: 0.6978 - acc: 0.5001 - val_loss: 0.6908 - val_acc: 0.6195
Epoch 52/300
1/1 [==============================] - 2s 2s/step - loss: 0.6949 - acc: 0.5139 - val_loss: 0.6907 - val_acc: 0.6195
Epoch 53/300
1/1 [==============================] - 2s 2s/step - loss: 0.6963 - acc: 0.5171 - val_loss: 0.6907 - val_acc: 0.6195
Epoch 54/300
1/1 [==============================] - 2s 2s/step - loss: 0.6972 - acc: 0.5280 - val_loss: 0.6906

In [5]:
a = [i for i in boosted_classifier.staged_predict_proba(X_test)]
len(a[0])
a

[array([[0.21392679, 0.7860732 ],
        [0.1088379 , 0.8911621 ],
        [0.10452993, 0.8954701 ],
        ...,
        [0.13649817, 0.86350185],
        [0.13144238, 0.86855763],
        [0.34920388, 0.6507961 ]], dtype=float32),
 array([[0.08810271, 0.9118973 ],
        [0.75865805, 0.24134195],
        [0.04114843, 0.9588516 ],
        ...,
        [0.05042784, 0.9495722 ],
        [0.04614718, 0.95385283],
        [0.10021538, 0.89978456]], dtype=float32),
 array([[0.04759111, 0.9524089 ],
        [0.3257683 , 0.67423177],
        [0.02678806, 0.97321194],
        ...,
        [0.06111384, 0.93888617],
        [0.31160578, 0.6883942 ],
        [0.21938969, 0.7806103 ]], dtype=float32),
 array([[0.1002076 , 0.8997924 ],
        [0.37915742, 0.62084264],
        [0.06648135, 0.93351865],
        ...,
        [0.11955763, 0.8804423 ],
        [0.36769423, 0.6323058 ],
        [0.28912866, 0.7108714 ]], dtype=float32),
 array([[0.14107035, 0.85892963],
        [0.39056277, 0.6094371

In [8]:
aa = [i for i in boosted_classifier.staged_predict(X_test)]
aa

[array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 0, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1])]

In [6]:
[accuracy_score(y_test,i) for i in boosted_classifier.staged_predict(X_test)]

[0.7611650485436893,
 0.7042071197411003,
 0.7294498381877023,
 0.7255663430420712,
 0.7281553398058253]

In [26]:
y_test

array([1, 0, 1, ..., 1, 1, 1])

In [ ]:
boosted_classifier.estimator_errors_

- .fit(new_x, new_y) : Adaboost accuracy : 0.7501618122977346, precision : 0.7742914979757085, recall : 0.8243534482758621, f1 : 0.7985386221294364, roc_auc : 0.7314635960990331
- .fit(new_x, new_y, **kwargs) : accuracy : 0.7430420711974111, precision : 0.7734294541709578, recall : 0.8092672413793104, f1 : 0.7909426013691417, roc_auc : 0.7263516109651819
- .fit(new_x, new_y, sample_weight=sample_weight) : 


In [5]:
y_pred_test = boosted_classifier.predict(X_test)

y_pred_test[y_pred_test>0.5]=1
y_pred_test[y_pred_test<=0.5]=0
precision = precision_score(y_test, y_pred_test)
recall = recall_score(y_test, y_pred_test)
f1 = f1_score(y_test, y_pred_test)
roc_auc = roc_auc_score(y_test, y_pred_test)
acc = accuracy_score(y_test, y_pred_test)

print(f'accuracy : {acc}, precision : {precision}, recall : {recall}, f1 : {f1}, roc_auc : {roc_auc}')

accuracy : 0.7430420711974111, precision : 0.7734294541709578, recall : 0.8092672413793104, f1 : 0.7909426013691417, roc_auc : 0.7263516109651819


## 모델저장

In [8]:
import pickle
import joblib

joblib.dump(boosted_classifier, './models/adaboost_lstm.pkl') 
reload_model = joblib.load('./models/adaboost_lstm.pkl') 

INFO:tensorflow:Assets written to: ram://4fc3a096-2c5e-42aa-8ed7-0d35dd718c6c/assets


INFO:tensorflow:Assets written to: ram://3f4d161d-ae84-45e1-b949-b9e25640463a/assets


INFO:tensorflow:Assets written to: ram://3f4d161d-ae84-45e1-b949-b9e25640463a/assets


INFO:tensorflow:Assets written to: ram://aea38500-abca-43c1-9f03-3d652beb35d0/assets


INFO:tensorflow:Assets written to: ram://aea38500-abca-43c1-9f03-3d652beb35d0/assets


...

INFO:tensorflow:Assets written to: ram://94288a7f-24f6-4f72-9a74-76d071b01a68/assets


INFO:tensorflow:Assets written to: ram://53037386-12b4-4385-b5cb-98f69aed3181/assets


INFO:tensorflow:Assets written to: ram://53037386-12b4-4385-b5cb-98f69aed3181/assets


INFO:tensorflow:Assets written to: ram://56c8a39b-d555-4124-a103-af0a24f06b89/assets


INFO:tensorflow:Assets written to: ram://56c8a39b-d555-4124-a103-af0a24f06b89/assets


## etc..

In [4]:
a = [i for i in boosted_classifier.staged_predict_proba(X_test)]
len(a[0])
a

[array([[0.14671701, 0.853283  ],
        [0.15134184, 0.84865814],
        [0.09548907, 0.9045109 ],
        ...,
        [0.12648392, 0.8735161 ],
        [0.10223823, 0.89776176],
        [0.23039924, 0.76960075]], dtype=float32),
 array([[0.8154902 , 0.18450987],
        [0.8216686 , 0.17833138],
        [0.0754172 , 0.9245828 ],
        ...,
        [0.20071441, 0.7992856 ],
        [0.06067883, 0.9393212 ],
        [0.0725329 , 0.9274671 ]], dtype=float32),
 array([[0.34155625, 0.6584438 ],
        [0.34451336, 0.6554866 ],
        [0.03459236, 0.9654076 ],
        ...,
        [0.08024294, 0.919757  ],
        [0.04676704, 0.95323294],
        [0.12316583, 0.8768342 ]], dtype=float32),
 array([[0.5092239 , 0.49077615],
        [0.60728794, 0.3927121 ],
        [0.16986866, 0.8301313 ],
        ...,
        [0.2854696 , 0.7145304 ],
        [0.20876725, 0.79123276],
        [0.24563938, 0.7543606 ]], dtype=float32),
 array([[0.53762466, 0.46237534],
        [0.3876752 , 0.6123248

In [8]:
[accuracy_score(y_test,i) for i in boosted_classifier.staged_predict(X_test)]

[0.7682847896440129,
 0.6220064724919094,
 0.7592233009708738,
 0.6517799352750809,
 0.7391585760517799,
 0.6660194174757281,
 0.7249190938511327,
 0.6724919093851133,
 0.7171521035598706,
 0.6964401294498382]

In [27]:
boosted_classifier.estimator_weights_

array([1., 1., 1., 1., 1.])

In [28]:
boosted_classifier.estimator_errors_

array([0.13636364, 0.18618371, 0.77088202, 0.80480714, 0.13529329])

In [100]:
y_pred_test

array([0, 0, 1, ..., 1, 1, 1])

In [101]:
boosted_classifier.estimators_

In [102]:
boosted_classifier.estimator_weights_

array([1., 1., 1., 1., 1.])

In [103]:
boosted_classifier.estimator_errors_

array([0.19136202, 0.19596451, 0.16152457, 0.14712343, 0.16450197])